Trend analysis of utilisation segments


Read in Data

In [1]:
import pandas as pd
import pdb
import mannKendall
import numpy as np

import plotly.tools as tls
tls.set_credentials_file(username='NTPlotly', api_key='yViUde2xMRvWglgycSg4') #my plotly credentials (please dont use they cost me $$)
import plotly.plotly as py
#from plotly.graph_objs import *
import plotly.graph_objs as go
from plotly import tools
#import cufflinks as cf
import matplotlib
from IPython.core.display import display #display inline package


In [2]:
def doAnalysis(data):
    
    
    #pdb.set_trace()
    
    allH = []
    
    allTrend = []
    
    allP = []
    
    allZ = []
    
    allSize = []
    
    allX = []
    
    allY = []
    
    allStart = []
    
    allFinish = []
    
    for i in range(66):
        
        #i = 1 
        
        for ii in range(66):
            
            #ii = 32
            
            sliced = data[(data['x'] == i) & (data['y'] == ii)]
            
            sliced['dates'] = pd.to_datetime(sliced['dates'])
            
            peakDates = sliced[(sliced['peaks']==True)]
            
            if peakDates.empty == False:
            
                #pdb.set_trace()           

                peak1 = peakDates['dates'].iloc[0]

                troughDates = sliced[(sliced['troughs']==True)]
                
                if troughDates.empty == False:

                    trough1 = troughDates['dates'].iloc[0]

                    diff = trough1 - peak1

                    diff = diff.days       

                    #print str(i) + '_' + str(ii)

                    if diff >0: 

                        for z in range(len(peakDates)):

                            #z = 4

                            peak = peakDates['dates'].iloc[z]

                            if z < len(troughDates):

                                #pdb.set_trace()

                                #print z

                                trough = troughDates['dates'].iloc[z]

                                slicedPeak = sliced[(sliced['dates'] > peak) & (sliced['dates'] <= trough)]

                                v = slicedPeak['fittedSpline'].as_matrix()

                                #print str(z)

                                if len(v)>5:

                                    h, trend, p, z = mannKendall.mmk_test(v)

                                    allH.append(h)

                                    allTrend.append(trend)

                                    allP.append(p)

                                    allZ.append(z)

                                    allX.append(i)

                                    allY.append(ii)

                                    allSize.append(len(v))
                                    
                                    allStart.append(peak)
                                    
                                    allFinish.append(trough)
                                    
                            

    #pdb.set_trace()

    df = pd.DataFrame()
    
    df['h'] = allH
    
    df['trend'] = allTrend
    
    df['p'] = allP
    
    df['z'] = allZ
    
    df['x'] = allX
    
    df['y'] = allY
    
    df['size'] = allSize
    
    df['start'] = allStart
    
    df['finish'] = allFinish
    
    return df
    
    

In [3]:
def doAll():
    
    timeSeriesList = pd.read_csv('data/list.txt',header=None)
    
    for i in range(len(timeSeriesList)):
        
        print i
        
        fileName = timeSeriesList.loc[i]
        
        fileName = fileName[0]
        
        
        fileName1 = 'data/' + fileName
        
        data = pd.read_csv(fileName1,header=0)        
        
        df = doAnalysis(data)
        
        trendSum = df['h'].sum()
        
        propT = (trendSum / float(len(df)))*100
        
        propF = 100 - propT
        
        
        
        #pdb.set_trace()
        
        
        
        #trendCnt = trend.sum()
        
        #pdb.set_trace()
        
        outName = 'outputs/' + fileName[:-4]+ 'MK.csv'
        
        df.to_csv(outName)
        
        
        
        #doPlot(df,data)
        
        #plotAll = go.Scatter(name = 'NPV', x=datesA, y=yA,mode = 'lines',line = dict(width=1,color = 'rgb(1,1,1)'),showlegend = lgnd)        

        
      
        #pdb.set_trace()

In [4]:
#doAll()

In [5]:
def doScatter(modelledData,trendData,dfV,title):
    
    plot = []
        
    df = dfV
    
    annotations = []
    
    
    
    #pdb.set_trace()
    
    modelledData = modelledData.set_index(['index'])
    
    for i in modelledData.index:
        
        if i == 0:
            
            a = go.Scatter(x=df['dates'], y=df['fittedSpline']-100.0,name='NPV Spline',mode = 'lines', line = dict(color = ('rgb(1, 1, 1)'),width = 1))
            
            b = go.Scatter(x=df['dates'], y=df['npv']-100.0,name='NPV',mode = 'markers',marker = dict(color = ('rgb(1, 1, 1)'),size = 2,symbol='circle'))
                 
            
            plot.append(a)
            
            plot.append(b)
        
        ind = modelledData.ix[i]
        
        #pdb.set_trace()
        
        minNpv = min(df['npv'])
        
        start = ind['sDate']
        
        end = ind['eDate']
        
        mask = (df['dates'] > start) & (df['dates'] <= end)
        
        sliced = df.loc[mask]
                        
        slicedLinStart = sliced.head(1)
        
        slicedLinEnd = sliced.tail(1)
        
        frames = [slicedLinStart,slicedLinEnd]
        
        result = pd.concat(frames)
        
        
                                       
        st = str(start)
        
        st=st[:-9]
        
        ed = str(end)
        
        ed=ed[:-9]
        
        
        slicedEnd = sliced['fittedSpline'].iloc[len(sliced)-1]
        
        #slicedEnd = slicedEnd-100
        
        sliced['fittedSplineBase'] = slicedEnd
        
        
        #pdb.set_trace()
        
        #sln = go.Scatter(x=result['dates'],y=result['npv'], mode='line',line = dict(color = ('rgb(2, 2, 2)'),width = 5,),name=False)
        trend = trendData['trend'].iloc[i]
        
        trend = round(trend,3)
        
        trend = 'Trend = ' + str(trend)
        
        #pdb.set_trace()
         
        if i == 0:
        
            
            #pdb.set_trace()
            
            '''
            
            slnFitted = go.Scatter(x=result['dates'],y=result['fittedSpline']-100.0, mode='lines',name='Slope',line = dict(color = ('rgb(1, 1, 1)'),width = 4),showlegend=True)
                  
            pl = go.Scatter(x=sliced['dates'],y=sliced['fittedSpline']-100.0, fill = 'tozeroy',name = 'Area under the curve (small-integral)',fillcolor=('rgb(220, 220, 220)'),mode='none',line=dict(width=0.5,
              color='rgb(1, 1, 1)'),showlegend=True)
            
            pll = go.Scatter(x=sliced['dates'],y=sliced['fittedSplineBase']-100.0, fill = 'tozeroy',name = 'Area under the curve (large-integral)',fillcolor=('rgb(240, 240, 240)'),mode='none',line=dict(width=0.5,
              color='rgb(1, 1, 1)'),showlegend=True)
                       
            
            plot.append(pl)
            
            plot.append(slnFitted)
            
            plot.append(pll)
            
            
            
            trace1b = go.Scatter(x=result['dates'].iloc[0],y=result['fittedSpline'].iloc[0]-100,name='Peaks',mode = 'markers', marker = dict(color = ('rgb(1, 1, 1)'),size = 13,symbol='circle'),showlegend=True)
    
            plot.append(trace1b)
        
            trace1b = go.Scatter(x=result['dates'].iloc[1],y=result['fittedSpline'].iloc[1]-100,name='Troughs',mode = 'markers', marker = dict(color = ('rgb(1, 1, 1)'),size = 13,symbol='triangle-up'),showlegend=True)
    
            plot.append(trace1b)
            
            '''
            
        
            #pdb.set_trace()
        
            start = pd.to_datetime(result['dates'].iloc[0])
            
            end = pd.to_datetime(result['dates'].iloc[1])
            
            df['dates']= pd.to_datetime(df['dates'])
        
        
            #midDate = a + (b - a)/2
            
            between = df[(df['dates'] >= start) & (df['dates'] <= end)]
            
            d = go.Scatter(x=between['dates'], y=between['fittedSpline']-100.0,name='Utilisation Stage',mode = 'lines', line = dict(color = ('rgb(1, 1, 1)'),width = 5),showlegend=True)
           
            plot.append(d)
                                    
            annotations.append(dict(x=result['dates'].iloc[0],y=result['fittedSpline'].iloc[0]-100,xref='x',yref='y',text = trend ,showarrow=True,arrowhead=7,ax=0,ay=-80,font=dict(size=20)))
        
            
        else:
            
            '''
            
            slnFitted = go.Scatter(x=result['dates'],y=result['fittedSpline']-100.0, mode='lines',name='Slope',line = dict(color = ('rgb(1, 1, 1)'),width = 4),showlegend=False)
      
                              
            pl = go.Scatter(x=sliced['dates'],y=sliced['fittedSpline']-100.0, fill = 'tozeroy',fillcolor=('rgb(220, 220, 220)'),mode='none',line=dict(width=0.5,
                  color='rgb(1, 1, 1)'),showlegend=False)
            
            pll = go.Scatter(x=sliced['dates'],y=sliced['fittedSplineBase']-100.0, fill = 'tozeroy',fillcolor=('rgb(240, 240, 240)'),mode='none',line=dict(width=0.5,
              color='rgb(1, 1, 1)'),showlegend=False)

            plot.append(pl)

            plot.append(slnFitted)
            
            plot.append(pll)
            
            
            
            trace1b = go.Scatter(x=result['dates'].iloc[0],y=result['fittedSpline'].iloc[0]-100,name='Peaks',mode = 'markers', marker = dict(color = ('rgb(1,1,1)'),size = 13,symbol='circle'),showlegend=False)
    
            plot.append(trace1b)
        
            trace1b = go.Scatter(x=result['dates'].iloc[1],y=result['fittedSpline'].iloc[1]-100,name='Troughs',mode = 'markers', marker = dict(color = ('rgb(1, 1, 1)'),size = 13,symbol='triangle-up'),showlegend=False)
    
            plot.append(trace1b)
            
            '''
        
            start = pd.to_datetime(result['dates'].iloc[0])
            
            end = pd.to_datetime(result['dates'].iloc[1])
            
            df['dates']= pd.to_datetime(df['dates'])
        
        
            #midDate = a + (b - a)/2
            
            between = df[(df['dates'] >= start) & (df['dates'] <= end)]
            
            d = go.Scatter(x=between['dates'], y=between['fittedSpline']-100.0,name='NPV Spline',mode = 'lines', line = dict(color = ('rgb(1, 1, 1)'),width = 5),showlegend=False)
           
            plot.append(d)
                             
            
            
            annotations.append(dict(x=result['dates'].iloc[0],y=result['fittedSpline'].iloc[0]-100,xref='x',yref='y',text = trend,showarrow=True,arrowhead=7,ax=0,ay=-80,font=dict(size=20)))
                
            
    
    #pdb.set_trace()
    
    #title = 'NPV time series'
            
    #layout = go.Layout(yaxis=dict(title='Non-Photosynthetic Vegetation Cover %'),xaxis=dict(title='Time'),font=dict(family='Arial, monospace', size=24, color = ('rgb(1, 1, 1)')))
    
    
    layout = go.Layout(title=title,yaxis=dict(title='Non-Photosynthetic Vegetation Cover %',range=[minNpv-100, 100],showgrid=False),xaxis=dict(title='Time',showgrid=False),font=dict(family='Arial, monospace', size=30, color = ('rgb(1, 1, 1)')),annotations = annotations)
      
    fig = go.Figure(data=plot, layout=layout)
    
    #url = py.iplot(fig,filename=filename)
    
    #pdb.set_trace()
    
    return fig

In [6]:
def trendSummary(data,fileName):
    
    results = pd.DataFrame()
        
    #pdb.set_trace()
    
    dataSize = len(data)
    
    trendCnt = data[(data['trend']>= 0)].count()
    
    noTrend = data[(data['trend']>= 0)]
    
    avgNoTrendLngth = noTrend['size'].mean()
    
    trendCnt = trendCnt['trend']
    
    propTrend = float(trendCnt) / float(dataSize) * 100
    
    avgP = data['p'].mean()
    
    avgZ = data['z'].mean()
        
       
    results['avgNoTrnd'] = [avgNoTrendLngth]
    
    results['propTrend'] = [propTrend]
    
    results['avgP'] = [avgP]
    
    results['avgZ'] = [avgZ]      
    
    results.to_csv(fileName)
    
    
    
    
    

In [7]:
def doPlotting():
    
    timeSeriesList = pd.read_csv('data/list.txt',header=None)
    
    names = ['Semi-arid to Arid Introduced Grassland','Arid to Semi-arid Chenopod Shrub-land and Grassland','Semi-arid to Arid Spinifex (Trioida sp.) Grassland','Sub-tropical to Semi-arid Grassland']
    
    #pdb.set_trace()
    
    for z in range(len(timeSeriesList)):
        
        print z
        
        #z = 2
        
        fileName = timeSeriesList.loc[z]
        
                
        fileName = fileName[0] 
        
        print fileName
        
        #print names[z]
        
        fileName1 = 'data/' + fileName
        
        #pdb.set_trace()
        
        fileName2 = fileName[17:]
        
        fileName2 = 'data/output_' + fileName2
        
        modelledData = pd.read_csv(fileName2,header=0)   
        
        dfV = pd.read_csv(fileName1,header=0)  
        
        trendFile = 'outputs/' + fileName[:-4]+ 'MK.csv'
        
        trendData = pd.read_csv(trendFile,header=0)
        
        trendOut = 'outputs/' + fileName[:-4] + 'trendSummary.csv'
        
        trendSumm = trendSummary(trendData,trendOut)
        
        #pdb.set_trace()
        
        for i in range(40,50):
            
            for ii in range(40,50):
        
                #i = 21

                #ii = 38  
                
                print 'i=' + str(i)
                
                print 'ii=' + str(ii)

                trendData1 = trendData[(trendData['x'] == i) & (trendData['y'] == ii)]

                if trendData1.empty == False:                   
                    

                    modelledData1 = modelledData[(modelledData['x'] == i) & (modelledData['y'] == ii)]
                    
                    #pdb.set_trace()
                    
                    if len(trendData1) == len(modelledData1):

                        dfV = dfV[(dfV['x'] == i) & (dfV['y'] == ii)]

                        #pdb.set_trace
                        
                        print 'hit'

                        fig = doScatter(modelledData1,trendData1,dfV,names[z])

                        break

                    #pdb.set_trace()
            if len(trendData1) == len(modelledData1):
                    
                    break
        
        #fig = doPlot(modelledData,trendData,dfV)
            
        
        filename1 = fileName[17:-5] + 'pltAA'
        
        #pdb.set_trace()
        
        filename1 = 'outputs/' + filename1
        
        #print filename1
    
        peakPlot = py.iplot(fig, filename=filename1)
        
        display(peakPlot)
        
        #pdb.set_trace()
    

In [8]:
doPlotting()

0
outputTimeSeries_13272792e02458094s.csv
i=40
ii=40
hit


/opt/conda/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:65: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



1
outputTimeSeries_13353356e02471525s.csv
i=40
ii=40
i=40
ii=41
i=40
ii=42
i=40
ii=43
i=40
ii=44
i=40
ii=45
hit


2
outputTimeSeries_13398721e02392138s.csv
i=40
ii=40
i=40
ii=41
i=40
ii=42
hit


3
outputTimeSeries_13513946e02036131s.csv
i=40
ii=40
i=40
ii=41
i=40
ii=42
i=40
ii=43
i=40
ii=44
i=40
ii=45
hit
